In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from airsim.collections import AirObject, AirEnv, RadarSystem, ControlPoint, Supervisor

In [2]:
air_objects = [
    AirObject(position=[-8,  0, 3], velocity=[ 1, 0, 0], acceleration=[0, 0, 0]),
    AirObject(position=[ 8,  0, 3], velocity=[-1, 0, 0], acceleration=[0, 0, 0]),
    AirObject(position=[-8,  2, 3], velocity=[ 1, 0, 0], acceleration=[0, 0, 0]),
    AirObject(position=[ 8,  2, 3], velocity=[-1, 0, 0], acceleration=[0, 0, 0]),
    AirObject(position=[-8, -2, 3], velocity=[ 1, 0, 0], acceleration=[0, 0, 0]),
    AirObject(position=[ 8, -2, 3], velocity=[-1, 0, 0], acceleration=[0, 0, 0])
]

radar_systems = [
    RadarSystem(position=[ 3, 0, 0], detection_radius=50, error=0.01),
    RadarSystem(position=[-3, 0, 0], detection_radius=50, error=0.01)
]

air_env = AirEnv()
for ao in air_objects:
    air_env.attach_air_object(ao)

control_point = ControlPoint()
for rs in radar_systems:
    rs.attach_air_environment(air_env)
    control_point.attach_radar_system(rs)

supervisor = Supervisor()
supervisor.attach_air_environment(air_env)
supervisor.attach_control_point(control_point)
supervisor.attach_radar_systems(radar_systems)

In [3]:
dt    =  0.005
t_min =  0.000
t_mid = 16.000 - dt
t_max = 16.000

In [4]:
air_env.set_public_ids(True)
supervisor.run(t_min, t_mid, dt)

air_env.set_public_ids(False)
supervisor.run(t_mid + dt, t_max, dt)

In [5]:
data = control_point.get_data()[['detection_time', 'radar_system_id', 'air_object_id', 'detection_error',
                                 'air_object_x', 'air_object_y', 'air_object_z']]
data

,detection_time,radar_system_id,air_object_id,detection_error,air_object_x,air_object_y,air_object_z
0,0.0,0.0,0,0.01,-8.002110,-0.001182,2.994241
1,0.0,0.0,1,0.01,8.002358,-0.006855,2.998438
2,0.0,0.0,2,0.01,-8.010949,1.998607,2.996671
3,0.0,0.0,3,0.01,7.991862,2.012634,2.990732
4,0.0,0.0,4,0.01,-8.014156,-1.997187,3.001646
...,...,...,...,...,...,...,...
38395,16.0,1.0,-1,0.01,-8.002417,0.022576,3.001743
38396,16.0,1.0,-1,0.01,8.009587,2.016010,3.002506
38397,16.0,1.0,-1,0.01,-8.019421,1.985009,2.992890
38398,16.0,1.0,-1,0.01,7.985947,-2.005823,3.001056


In [6]:
train = data[data['air_object_id'] >= 0]
x_train, y_train = [], []

for t in np.linspace(t_min, t_mid, round((t_mid - t_min)/dt + 1)):
    data_t = train[(train['detection_time'] >= (t - 0.1 * dt)) & (train['detection_time'] <= (t + 0.1 * dt))].sample(frac=1)
    x_t = data_t[['air_object_x', 'air_object_y', 'air_object_z', 'detection_error']]
    y_t = data_t['air_object_id']
    if len(x_t) != 0:
        x_t, y_t = x_t.to_numpy(), y_t.to_numpy()
        x_train.append(x_t)
        y_train.append(y_t)

x_train, y_train = np.moveaxis(np.array(x_train), 0, 1), np.moveaxis(np.array(y_train), 0, 1)
print(x_train.shape, y_train.shape)

(12, 3199, 4) (12, 3199)


In [7]:
test = data[data['air_object_id'] == -1]
x_test = []

for t in np.linspace(t_mid + dt, t_max, round((t_max - t_mid)/dt)):
    data_t = test[(test['detection_time'] >= (t - 0.1 * dt)) & (test['detection_time'] <= (t + 0.1 * dt))].sample(frac=1)
    x_t = data_t[['air_object_x', 'air_object_y', 'air_object_z', 'detection_error']]
    if len(x_t) != 0:
        x_t = x_t.to_numpy()
        x_test.append(x_t)

x_test = np.moveaxis(np.array(x_test), 0, 1)
print(x_test.shape)

(12, 1, 4)


In [8]:
import tensorflow as tf
from tensorflow import keras

# Prepare your data
# Assume X_train and y_train are your training data

# Define your model
model = keras.Sequential([
    keras.layers.SimpleRNN(units=32),#, input_shape=(x_train.shape[1], x_train.shape[2])),
    keras.layers.Dense(units=1)
])

# Compile your model
model.compile(loss='mse', optimizer='adam')

# Train your model
history = model.fit(x_train, y_train, epochs=50, batch_size=12, validation_split=0.2)

# Evaluate your model
# loss = model.evaluate(x_test, y_test)

# Make predictions
y_test = model.predict(x_test)

print(np.round(x_test, 1), np.round(y_test, 4))
print(x_test.shape, y_test.shape)

/Users/arsen-osipyan/Desktop/air_objects_identification/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Epoch 1/50
1/1 [==============================] - 0s 419ms/step - loss: 14.4727 - val_loss: 12.0987
Epoch 2/50
1/1 [==============================] - 0s 107ms/step - loss: 14.0676 - val_loss: 12.0752
Epoch 3/50
1/1 [==============================] - 0s 108ms/step - loss: 13.6706 - val_loss: 12.0479
Epoch 4/50
1/1 [==============================] - 0s 107ms/step - loss: 13.2823 - val_loss: 12.0189
Epoch 5/50
1/1 [==============================] - 0s 109ms/step - loss: 12.9036 - val_loss: 11.9886
Epoch 6/50
1/1 [==============================] - 0s 108ms/step - loss: 12.5356 - val_loss: 11.9568
Epoch 7/50
1/1 [==============================] - 0s 109ms/step - loss: 12.1793 - val_loss: 11.9227
Epoch 8/50
1/1 [==============================] - 0s 108ms/step - loss: 11.8356 - val_loss: 11.8859
Epoch 9/50
1/1 [==============================] - 0s 108ms/step - loss: 11.5052 - val_loss: 11.8456
Epoch 10/50
1/1 [==============================] - 0s 107ms/step - loss: 11.1884 - val_loss: 11.8015

In [13]:
from sklearn.cluster import KMeans

# generate some random data
data = y_test.T[0]
print(data * 100)
print(len(data))

# specify the number of clusters
n_clusters = int(len(data) / len(radar_systems))

# create a KMeans object with the specified number of clusters
kmeans = KMeans(n_clusters=n_clusters)

# fit the data to the KMeans model
kmeans.fit(data.reshape(-1, 1))

# get the labels for each data point
labels = kmeans.labels_

# print the labels for each data point
print(labels)


[23.364399   3.7253141 25.560024  32.500298  21.221916   3.7204638
 21.331112  25.423038  23.428793  32.76663   25.344622  25.58589  ]
12
[4 2 5 1 0 2 0 3 4 1 3 5]


/Users/arsen-osipyan/Desktop/air_objects_identification/venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
raise RuntimeError()

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from airsim.collections import AirObject, AirEnv, RadarSystem, ControlPoint, Supervisor

In [2]:
air_objects = [
    AirObject(position=[-8,  0, 3], velocity=[ 1, 0, 0], acceleration=[0, 0, 0]),
    AirObject(position=[ 8,  0, 3], velocity=[-1, 0, 0], acceleration=[0, 0, 0]),
    AirObject(position=[-8,  2, 3], velocity=[ 1, 0, 0], acceleration=[0, 0, 0]),
    AirObject(position=[ 8,  2, 3], velocity=[-1, 0, 0], acceleration=[0, 0, 0]),
    AirObject(position=[-8, -2, 3], velocity=[ 1, 0, 0], acceleration=[0, 0, 0]),
    AirObject(position=[ 8, -2, 3], velocity=[-1, 0, 0], acceleration=[0, 0, 0]),
    AirObject(position=[ 0, -16, 3], velocity=[0, 2, 0], acceleration=[0, 0, 0])
]

radar_systems = [
    RadarSystem(position=[ 3, 0, 0], detection_radius=10, error=0.01),
    RadarSystem(position=[-3, 0, 0], detection_radius=10, error=0.01)
]

air_env = AirEnv()
for ao in air_objects:
    air_env.attach_air_object(ao)

control_point = ControlPoint()
for rs in radar_systems:
    rs.attach_air_environment(air_env)
    control_point.attach_radar_system(rs)

supervisor = Supervisor()
supervisor.attach_air_environment(air_env)
supervisor.attach_control_point(control_point)
supervisor.attach_radar_systems(radar_systems)

In [3]:
dt    =  0.01
t_min =  0.00
t_max = 16.00

In [4]:
air_env.set_public_ids(True)
supervisor.run(t_min, t_max, dt)

In [5]:
data = control_point.get_data()[['detection_time', 'radar_system_id', 'air_object_id', 'detection_error',
                                 'air_object_x', 'air_object_y', 'air_object_z']]
data

,detection_time,radar_system_id,air_object_id,detection_error,air_object_x,air_object_y,air_object_z
1,0.0,0.0,1,0.01,7.988815,-0.001720,3.007521
3,0.0,0.0,3,0.01,8.007425,1.986412,3.015114
5,0.0,0.0,5,0.01,7.998104,-2.008972,2.999661
3,0.0,1.0,0,0.01,-8.011195,-0.000324,3.004036
5,0.0,1.0,2,0.01,-8.007900,2.008299,3.019724
...,...,...,...,...,...,...,...
19086,16.0,0.0,2,0.01,7.993629,2.000060,3.010934
19088,16.0,0.0,4,0.01,7.984029,-2.009248,2.999929
19088,16.0,1.0,1,0.01,-7.992748,0.005074,3.007931
19090,16.0,1.0,3,0.01,-7.993011,1.997265,2.995642


In [9]:
max_batch_size = -1

for t in np.linspace(t_min, t_max, round((t_max - t_min)/dt + 1)):
    d = data[(data['detection_time'] > (t - 0.1 * dt)) & (data['detection_time'] < (t + 0.1 * dt))]
    if d.shape[0] > max_batch_size:
        max_batch_size = d.shape[0]

print(f'max batch size - {max_batch_size}')

max batch size - 14


In [19]:
for t in np.linspace(t_min, t_max, round((t_max - t_min)/dt + 1)):
    d = data[(data['detection_time'] > (t - 0.1 * dt)) & (data['detection_time'] < (t + 0.1 * dt))]

    row = pd.Series({
        'detection_time':     t,
        'radar_system_id': -1.0,
        'air_object_id':   -1.0,
        'detection_error':  0.0,
        'air_object_x':     0.0,
        'air_object_y':     0.0,
        'air_object_z':     0.0
    })
    delta = max_batch_size - len(d)
    print(delta)

    for _ in range(delta):
        # data.loc[len(data)] = row.copy()
        data = pd.concat([data.loc[:], row]).reset_index(drop=True)

8
(19098, 8)
(19105, 8)
(19112, 8)
(19119, 8)
(19126, 8)
(19133, 8)
(19140, 8)
(19147, 8)
8
(19154, 8)
(19161, 8)
(19168, 8)
(19175, 8)
(19182, 8)
(19189, 8)
(19196, 8)
(19203, 8)
8
(19210, 8)
(19217, 8)
(19224, 8)
(19231, 8)
(19238, 8)
(19245, 8)
(19252, 8)
(19259, 8)
8
(19266, 8)
(19273, 8)
(19280, 8)
(19287, 8)
(19294, 8)
(19301, 8)
(19308, 8)
(19315, 8)
8
(19322, 8)
(19329, 8)
(19336, 8)
(19343, 8)
(19350, 8)
(19357, 8)
(19364, 8)
(19371, 8)
8
(19378, 8)
(19385, 8)
(19392, 8)
(19399, 8)
(19406, 8)
(19413, 8)
(19420, 8)
(19427, 8)
8
(19434, 8)
(19441, 8)
(19448, 8)
(19455, 8)
(19462, 8)
(19469, 8)
(19476, 8)
(19483, 8)
8
(19490, 8)
(19497, 8)
(19504, 8)
(19511, 8)
(19518, 8)
(19525, 8)
(19532, 8)
(19539, 8)
8
(19546, 8)
(19553, 8)
(19560, 8)
(19567, 8)
(19574, 8)
(19581, 8)
(19588, 8)
(19595, 8)
8
(19602, 8)
(19609, 8)
(19616, 8)
(19623, 8)
(19630, 8)
(19637, 8)
(19644, 8)
(19651, 8)
8
(19658, 8)
(19665, 8)
(19672, 8)
(19679, 8)
(19686, 8)
(19693, 8)
(19700, 8)
(19707, 8)
8
(19714, 

In [8]:
min_batch_size = len(data) + 1
max_batch_size = -1

for t in np.linspace(t_min, t_max, round((t_max - t_min)/dt + 1)):
    d = data[(data['detection_time'] > (t - 0.1 * dt)) & (data['detection_time'] < (t + 0.1 * dt))]

    print(d.shape)
    
    if d.shape[0] > max_batch_size:
        max_batch_size = d.shape[0]
    if d.shape[0] < min_batch_size:
        min_batch_size = d.shape[0]

print(min_batch_size, max_batch_size)

(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)